## Tobigs 3주차 DecisionTree 과제1
#### 15기 이윤정
---

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [3]:
def get_gini(df, label):
    gini = 1
    
    for i in range(0, len(df[label].unique())):
        mat = df.loc[df[label] == df[label].unique()[i]]
        p = len(mat)/len(df)
        gini -= np.square(p)    
    
    return gini    

In [4]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [5]:
from itertools import combinations

def get_binary_split(df, attribute):
    result = []
    
    for i in range(1, len(df[attribute].unique())):
        c = combinations(df[attribute].unique(), i)
        result += list(c)
        
    return result

In [6]:
get_binary_split(pd_data,"age")  #튜플은 단일 원소일 때 원소 뒤에 ,(comma)를 출력함!

[('youth',),
 ('middle_aged',),
 ('senior',),
 ('youth', 'middle_aged'),
 ('youth', 'senior'),
 ('middle_aged', 'senior')]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [7]:
def get_attribute_gini_index(df, attribute, label):
    split = get_binary_split(df, attribute)
    gini = []
    gini_i = 0
        
    for i in range(0, len(split)) :
        inner = df[df[attribute].isin(split[i])]
        outter = df[df[attribute].isin(split[i]) == False]
        gini_i = (len(inner)/len(df))*get_gini(inner, label)+(len(outter)/len(df))*get_gini(outter, label)
            
        gini.append(gini_i)
            
    result = {split: gini for split, gini in zip(split, gini)}    
        
    return result

In [8]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")  #튜플은 단일 원소일 때 원소 뒤에 ,(comma)를 출력함!

{('youth',): 0.3936507936507937,
 ('middle_aged',): 0.35714285714285715,
 ('senior',): 0.45714285714285713,
 ('youth', 'middle_aged'): 0.45714285714285713,
 ('youth', 'senior'): 0.35714285714285715,
 ('middle_aged', 'senior'): 0.3936507936507937}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

In [9]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())

(('middle_aged',), 0.35714285714285715)

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [10]:
##문제1 답안
get_binary_split(pd_data,"income")

[('high',),
 ('medium',),
 ('low',),
 ('high', 'medium'),
 ('high', 'low'),
 ('medium', 'low')]

In [11]:
##문제2 답안
def variable_imp(df, target):
    feature = [i for i in df.columns if i != target and i !="class_buys_computer"]
    label = ""
    col = ""
    minimum = 9999
    
    for j in feature: 
        le, gini = min(get_attribute_gini_index(df, j, "class_buys_computer").items())

        minimum = minimum if minimum<=gini else gini
        label = le if minimum == gini else label
        col = j if minimum == gini else col
        
    return col, label, minimum

In [12]:
col, label, minimum = variable_imp(pd_data, "class_buys_computer")
print(col,">>",label,">>",minimum)

age >> ('middle_aged',) >> 0.35714285714285715


In [13]:
##문제3 답안
#split
inter = pd_data[pd_data[col].isin(label)]
diff = pd_data[~pd_data[col].isin(label)]

col_i, label_i, minimum_i = variable_imp(inter, "age")
print("Split Set 1:")
print(col_i,">>",label_i,">>",minimum_i)

col_o, label_o, minimum_o = variable_imp(diff, "age")
print("\nSplit Set 2:")
print(col_o,">>",label_o,">>",minimum_o)

Split Set 1:
credit_rating >> ('excellent',) >> 0.0

Split Set 2:
student >> ('no',) >> 0.31999999999999984
